### <span style="color:red">This notebook takes a very long time to run! Do not re-run unless REALLY necessary!</span>

## Parse FAERS Data to Spreadsheets
#### *(so that this piece of code does not need to be run again... ever... ;-) )*

In [2]:
import os
import glob
import pandas as pd

FAERS is the FDA Adverse Event Reporting System. Adverse effects reported during different time periods can be downloaded as ASCII-formatted relational databases. Data from each time period are stored in their own subdirectories. For example, the subdirectory `faers_ascii_2014q2` contains a couple of documentation files and a subdirectory `ascii`, and in `ascii` there are several databases saved as `????14Q2.txt`.

The code digs into these subdirectories to read the data. All data is merged into one (VERY!) big dataframe. There is a massive amount of data, so creating this dataframe can be very time consuming.

In [3]:
root_dir = "/Users/gogrean/Documents/Insight_Fellowship/Research/Mental_Health/NHANES_Survey/"
os.chdir(root_dir)
faers_data_dir = "data/FAERS/"
faers_data_subdirs = [d[0] for d in os.walk(faers_data_dir) if d[0][-5:]=='ascii']

In [105]:
df_drugs_list = []
df_reactions_list = []
for d in faers_data_subdirs:
    os.chdir(root_dir + d)
    for fname_drugs, fname_reactions in zip(glob.glob("DRUG*.txt"),glob.glob("REAC*.txt")):
        with open(fname_drugs, 'r') as f:
            df_drugs_list.append(pd.read_table(f, sep='$'))
        with open(fname_reactions, 'r') as f:
            df_reactions_list.append(pd.read_table(f, sep='$'))
df_drugs = pd.concat(df_drugs_list, ignore_index=True)
df_reactions = pd.concat(df_reactions_list, ignore_index=True)

# drop a few columns in a desperate attempt to speed things up
df_drugs.drop(labels=['cum_dose_chr', 'cum_dose_unit', 'dechal', 'dose_amt',
                      'dose_form', 'dose_freq', 'dose_unit', 'dose_vbm', 'drug_seq', 
                      'exp_dt', 'lot_num', 'nda_num', 'rechal', 'role_cod', 'route', 'val_vbm'],
              axis=1, inplace=True)
df_reactions.drop(labels=['drug_rec_act'], axis=1, inplace=True)

/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (11,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (11,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda/lib

In [111]:
# just making a point that handling these takes time...
print('Summary of the df_drugs dataframe...\n')
print(df_drugs.info())
print('\n\nSummary of the df_reactions dataframe...\n')
print(df_reactions.info())

Summary of the df_drugs dataframe...

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14884310 entries, 0 to 14884309
Data columns (total 4 columns):
caseid       int64
drugname     object
primaryid    int64
prod_ai      object
dtypes: int64(2), object(2)
memory usage: 567.8+ MB
None


Summary of the df_reactions dataframe...

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12618453 entries, 0 to 12618452
Data columns (total 3 columns):
caseid       int64
primaryid    int64
pt           object
dtypes: int64(2), object(1)
memory usage: 385.1+ MB
None


Since I only care about drugs used to treat psychiatric conditions, I exclude all the other rows. This makes future handling of the dataframes a little quicker. To do this easier, I check the generic drug names against entries in `brand_to_generic_drug_names.csv`. Some of the generic names are slightly different, with the ones in the FDA database being typically more precise. For simplicity though, I changed all the generic drug names to match the simpler name that I have in my file.

In [112]:
os.chdir(root_dir + "data/")

df_psycho_drugs = pd.read_csv("brand_to_generic_drug_names.csv")
generic_drug_names = df_psycho_drugs["Generic Name"].unique()

bad_indices = []
for index, row in df_drugs.iterrows():
    # TODO:
    # In some cases, the generic name is N/A.
    # Ignore for now since there is plenty of data, but this
    # could be handled better in the future...
    if isinstance(df_drugs.at[index,"prod_ai"], float):
        bad_indices.append(index)
        continue
    # in the case of drugs that are combos of several others, the 
    # names of the individual drugs are separated by "\"
    chem_combo = df_drugs.at[index,"prod_ai"].strip().split("\\")
    simpler_chems = []
    # Super hacky, but I only keep the first part of drug names that
    # have multiple words. This is enough to match the drug names I 
    # have with the more precise ones in the FDA database. So something 
    # like "DOXYCYCLINE HYCLATE" will become "DOXYCYCLINE".
    for chem in chem_combo:
        if len(chem):
            simpler_chems.append(chem.split()[0])
    # For a match when it comes to composite drugs, I separate them by 
    # a "; ", as I have them already in brand_to_generic_drug_names.csv.
    simpler_chem_combo = "; ".join(simpler_chems)
    # And finally, if the name of the drug listed in a row is not in the 
    # list of drugs used to treat psychiatric conditions, that row will 
    # be ignored.
    if simpler_chem_combo in generic_drug_names:
        df_drugs.at[index, 'simple_name'] = simpler_chem_combo
    else:
        bad_indices.append(index)

filtered_df_fda_drugs = df_drugs.drop(bad_indices).reset_index(drop=True)

KeyboardInterrupt: 

In [24]:
# Temporarily save the data, to avoid having to run the first bits of code 
# again in case this notebook needs to be re-run/debugged...
os.chdir(root_dir + "data/")
filtered_df_fda_drugs.to_csv("filtered_fda_drug_reports.csv", index=False)

In [5]:
# And now load the data. Not needed if it is all run in one piece, but useful 
# if the notebook needs to be re-run/debugged in multiple iterations.
os.chdir(root_dir + "data/")
df_fda_drugs_reported = pd.read_csv("filtered_fda_drug_reports.csv", header=0)

A list of the adverse effects reported for each entry ID is appended to the row with that ID in the newly-created dataframe. Just this part of the code takes about 2 hours to run...

In [73]:
from time import gmtime, strftime

good_pids = list(df_fda_drugs_reported['primaryid'].unique())
filtered_df_reactions = df_reactions[df_reactions.primaryid.isin(good_pids)]

i = 0
stored_effects_by_pid = {}

print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))
for index, row in df_fda_drugs_reported.iterrows():
    pid = df_fda_drugs_reported.at[index, 'primaryid']
    if pid not in stored_effects_by_pid:
        effect = "; ".join(df_reactions[df_reactions['primaryid']==row['primaryid']]['pt'])
        df_fda_drugs_reported.at[index, 'adverse_effects'] = effect
        stored_effects_by_pid[pid] = effect
    else:
        df_fda_drugs_reported.at[index, 'adverse_effects'] = stored_effects_by_pid[pid]
    if index % 10000 == 0:
        print('Done with the first... %i' %index)
                                
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

2018-01-17 21:25:43
Done with the first... 0
Done with the first... 10000
Done with the first... 20000
Done with the first... 30000
Done with the first... 40000
Done with the first... 50000
Done with the first... 60000
Done with the first... 70000
Done with the first... 80000
Done with the first... 90000
Done with the first... 100000
Done with the first... 110000
Done with the first... 120000
Done with the first... 130000
Done with the first... 140000
Done with the first... 150000
Done with the first... 160000
Done with the first... 170000
Done with the first... 180000
Done with the first... 190000
Done with the first... 200000
Done with the first... 210000
Done with the first... 220000
Done with the first... 230000
Done with the first... 240000
Done with the first... 250000
Done with the first... 260000
Done with the first... 270000
Done with the first... 280000
Done with the first... 290000
Done with the first... 300000
Done with the first... 310000
Done with the first... 320000
Done

In [77]:
# same reason as above to save the data...
os.chdir(root_dir + "data/")
df_fda_drugs_reported.to_csv("filtered_fda_drug_reports.csv", index=False)

In [78]:
# and then reload it...
os.chdir(root_dir + "data/")
df_fda_drugs_reported = pd.read_csv("filtered_fda_drug_reports.csv", header=0)

TODO: Function for each med. This was just to test the code...

In [103]:
import collections

parnate = df_fda_drugs_reported[df_fda_drugs_reported['simple_name'].str.lower() == 'tranylcypromine']

a = [y for x in parnate['adverse_effects'].str.split("; ") for y in x]
counter=collections.Counter(a)
[(eff, p/len(counter)) for (eff, p) in counter.most_common(20)]

[('Drug interaction', 0.11835748792270531),
 ('Depression', 0.07971014492753623),
 ('Headache', 0.07729468599033816),
 ('Anxiety', 0.07004830917874397),
 ('Condition aggravated', 0.07004830917874397),
 ('Nausea', 0.06521739130434782),
 ('Atrioventricular septal defect', 0.06521739130434782),
 ('Weight increased', 0.06521739130434782),
 ('Vomiting', 0.06280193236714976),
 ('Dizziness', 0.06038647342995169),
 ('Dysmorphism', 0.06038647342995169),
 ('Suicidal ideation', 0.06038647342995169),
 ('Hypoaesthesia', 0.06038647342995169),
 ('Paraesthesia', 0.057971014492753624),
 ('Foetal exposure during pregnancy', 0.057971014492753624),
 ('Muscle spasms', 0.057971014492753624),
 ('Hot flush', 0.057971014492753624),
 ('Hyperhidrosis', 0.05555555555555555),
 ('Serotonin syndrome', 0.05555555555555555),
 ('Anger', 0.05314009661835749)]

In [102]:
import collections

nardil = df_fda_drugs_reported[df_fda_drugs_reported['simple_name'].str.lower() == 'phenelzine']

a = [y for x in nardil['adverse_effects'].str.split("; ") for y in x]
counter=collections.Counter(a)
[(eff, p/len(counter)) for (eff, p) in counter.most_common(20)]

[('Drug ineffective', 0.14072494669509594),
 ('Drug interaction', 0.13859275053304904),
 ('Dizziness', 0.12153518123667377),
 ('Headache', 0.1044776119402985),
 ('Malaise', 0.1044776119402985),
 ('Serotonin syndrome', 0.1023454157782516),
 ('Anxiety', 0.09381663113006397),
 ('Fatigue', 0.09168443496801706),
 ('Hypertension', 0.07249466950959488),
 ('Intentional product misuse', 0.06609808102345416),
 ('Drug withdrawal syndrome', 0.06183368869936034),
 ('Fall', 0.057569296375266525),
 ('Product quality issue', 0.05543710021321962),
 ('Weight increased', 0.05543710021321962),
 ('Insomnia', 0.053304904051172705),
 ('Depression', 0.053304904051172705),
 ('Orthostatic hypotension', 0.053304904051172705),
 ('Balance disorder', 0.0511727078891258),
 ('Paraesthesia', 0.04904051172707889),
 ('Nervousness', 0.046908315565031986)]